<a href="https://colab.research.google.com/github/ribka-5/Machine-Learning/blob/main/Decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving PlayTennis.csv to PlayTennis (5).csv


In [ ]:
df = pd.read_csv("PlayTennis.csv")
df.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [ ]:
def calc_total_entropy(train_data, label, class_list):
 total_row = train_data.shape[0]
 total_entr = 0

 for c in class_list:
   total_class_count = train_data[train_data[label] == c].shape[0]
   total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row)
   total_entr += total_class_entr

 return total_entr

In [ ]:
def calc_entropy(feature_value_data, label, class_list):
 class_count = feature_value_data.shape[0]
 entropy = 0

 for c in class_list:
   label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]
   entropy_class = 0
   if label_class_count != 0:
     probability_class = label_class_count/class_count
     entropy_class = - probability_class * np.log2(probability_class)
     entropy += entropy_class
 return entropy

In [ ]:
def calc_info_gain(feature_name, train_data, label, class_list):
 feature_value_list = train_data[feature_name].unique()
 total_row = train_data.shape[0]
 feature_info = 0.0

 for feature_value in feature_value_list:
   feature_value_data = train_data[train_data[feature_name] == feature_value]
   feature_value_count = feature_value_data.shape[0]
   feature_value_entropy = calc_entropy(feature_value_data,label, class_list)
   feature_value_probability = feature_value_count/total_row
   feature_info += feature_value_probability * feature_value_entropy

 return calc_total_entropy(train_data, label, class_list) - feature_info

In [ ]:
def find_most_informative_feature(train_data, label, class_list):
 feature_list = train_data.columns.drop(label)
 max_info_gain = -1
 max_info_feature = None

 for feature in feature_list:
   feature_info_gain = calc_info_gain(feature, train_data,label, class_list)
   if max_info_gain < feature_info_gain:
     max_info_gain = feature_info_gain
     max_info_feature = feature
 return max_info_feature

In [ ]:
def generate_sub_tree(feature_name, train_data, label, class_list):
 feature_value_count_dict = train_data[feature_name].value_counts(sort=False)
 tree = {}

 for feature_value, count in feature_value_count_dict.iteritems():
   feature_value_data = train_data[train_data[feature_name] == feature_value]
   assigned_to_node = False
   for c in class_list:
     class_count = feature_value_data[feature_value_data[label] == c].shape[0]
     if class_count == count:
       tree[feature_value] = c
       train_data = train_data[train_data[feature_name] != feature_value]
       assigned_to_node = True

   if not assigned_to_node:
      tree[feature_value] = "?"
 return tree, train_data

In [ ]:
def make_tree(root, prev_feature_value, train_data, label,
class_list):
 if train_data.shape[0] != 0:
   max_info_feature = find_most_informative_feature(train_data, label, class_list)
   tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list)
   next_root = None

   if prev_feature_value != None:
     root[prev_feature_value] = dict()
     root[prev_feature_value][max_info_feature] = tree
     next_root = root[prev_feature_value][max_info_feature]
   else:
     root[max_info_feature] = tree
     next_root = root[max_info_feature]
     for node, branch in list(next_root.items()):
      if branch == "?":
        feature_value_data = train_data[train_data[max_info_feature] == node]
        make_tree(next_root, node, feature_value_data, label, class_list)

In [ ]:
def id3(df, label):
 train_data = df.copy()
 tree = {}
 class_list = train_data[label].unique()
 make_tree(tree, None, train_data, label, class_list)
 return tree

In [ ]:
def predict(tree, instance):
 if not isinstance(tree, dict):
   return tree
 else:
   root_node = next(iter(tree))
   feature_value = instance[root_node]
   if feature_value in tree[root_node]:
     return predict(tree[root_node][feature_value], instance)
   else:
     return None

In [ ]:
def evaluate(tree, test_data_m, label):
 correct_preditct = 0
 wrong_preditct = 0
 for index, row in test_data_m.iterrows():
   result = predict(tree, test_data_m.iloc[index])
   if result == test_data_m[label].iloc[index]:
     correct_preditct += 1
   else:
    wrong_preditct += 1
    accuracy = correct_preditct / (correct_preditct + wrong_preditct)
 return accuracy

In [ ]:
tree = id3(df, 'Play Tennis')
tree

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

In [ ]:
evaluate(tree, df, 'Play Tennis')

1.0

Latihan 2

Lakukan prediksi dan evaluasi terhadap dataset playingtennis di atas berdasarkan dataset
yang baru. Akses dataset test berikut:

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving PlayTennisTest.csv to PlayTennisTest (4).csv


In [ ]:
df1 = pd.read_csv("PlayTennisTest.csv")
df1.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Overcast,Mild,Normal,Weak,Yes
1,Sunny,Mild,Normal,Weak,Yes
2,Rain,Cool,High,Weak,No
3,Rain,Cool,High,Strong,No


In [ ]:
def calc_total_entropy(train_data, label, class_list):
 total_row = train_data.shape[0]
 total_entr = 0

 for c in class_list:
   total_class_count = train_data[train_data[label] == c].shape[0]
   total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row)
   total_entr += total_class_entr

 return total_entr

In [ ]:
def calc_entropy(feature_value_data, label, class_list):
 class_count = feature_value_data.shape[0]
 entropy = 0

 for c in class_list:
   label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]
   entropy_class = 0
   if label_class_count != 0:
     probability_class = label_class_count/class_count
     entropy_class = - probability_class * np.log2(probability_class)
     entropy += entropy_class
 return entropy

In [ ]:
def calc_info_gain(feature_name, train_data, label, class_list):
 feature_value_list = train_data[feature_name].unique()
 total_row = train_data.shape[0]
 feature_info = 0.0

 for feature_value in feature_value_list:
   feature_value_data = train_data[train_data[feature_name] == feature_value]
   feature_value_count = feature_value_data.shape[0]
   feature_value_entropy = calc_entropy(feature_value_data,label, class_list)
   feature_value_probability = feature_value_count/total_row
   feature_info += feature_value_probability * feature_value_entropy

 return calc_total_entropy(train_data, label, class_list) - feature_info

In [ ]:
def find_most_informative_feature(train_data, label, class_list):
 feature_list = train_data.columns.drop(label)
 max_info_gain = -1
 max_info_feature = None

 for feature in feature_list:
   feature_info_gain = calc_info_gain(feature, train_data,label, class_list)
   if max_info_gain < feature_info_gain:
     max_info_gain = feature_info_gain
     max_info_feature = feature
 return max_info_feature

In [ ]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False)
    tree = {}

    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value]

        assigned_to_node = False
        for c in class_list:
            class_count = feature_value_data[feature_value_data[label] == c].shape[0]

            if class_count == count:
                tree[feature_value] = c
                train_data = train_data[train_data[feature_name] != feature_value]
                assigned_to_node = True
        if not assigned_to_node:
            tree[feature_value] = "?"

    return tree, train_data

In [ ]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0:
        max_info_feature = find_most_informative_feature(train_data, label, class_list)
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list)
        next_root = None

        if prev_feature_value != None:
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else:
            root[max_info_feature] = tree
            next_root = root[max_info_feature]

        for node, branch in list(next_root.items()):
            if branch == "?":
                feature_value_data = train_data[train_data[max_info_feature] == node]
                make_tree(next_root, node, feature_value_data, label, class_list)

In [ ]:
def id3(df1, label):
 train_data = df.copy()
 tree = {}
 class_list = train_data[label].unique()
 make_tree(tree, None, train_data, label, class_list)
 return tree

In [ ]:
def predict(tree, instance):
 if not isinstance(tree, dict):
   return tree
 else:
   root_node = next(iter(tree))
   feature_value = instance[root_node]
   if feature_value in tree[root_node]:
     return predict(tree[root_node][feature_value], instance)
   else:
     return None

In [ ]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows():
        result = predict(tree, test_data_m.iloc[index])
        if result == test_data_m[label].iloc[index]:
            correct_preditct += 1
        else:
            wrong_preditct += 1
    accuracy = correct_preditct / (correct_preditct + wrong_preditct)
    return accuracy

In [ ]:
tree = id3(df1, 'Play Tennis')
tree

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

In [ ]:
evaluate(tree, df1, 'Play Tennis')

0.75

Latihan 3

Buatlah model pohon keputusan dengan algoritma ID3 seperti di atas, dengan menggunakan
dataset berikut:

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving datacovid19.txt to datacovid19 (4).txt


In [ ]:
df2 = pd.read_csv("datacovid19.txt", sep="\t", encoding="utf-16")
df2.head()

,Demam,Batuk,Gangguan Pernapasan,Terinfeksi
0,Ya,Ya,Ya,Ya
1,Ya,Ya,Tidak,Tidak
2,Ya,Tidak,Ya,Ya
3,Ya,Ya,Ya,Ya
4,Ya,Tidak,Ya,Ya


In [ ]:
def calc_total_entropy(train_data, label, class_list):
 total_row = train_data.shape[0]
 total_entr = 0

 for c in class_list:
   total_class_count = train_data[train_data[label] == c].shape[0]
   total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row)
   total_entr += total_class_entr

 return total_entr

In [ ]:
def calc_entropy(feature_value_data, label, class_list):
 class_count = feature_value_data.shape[0]
 entropy = 0

 for c in class_list:
   label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]
   entropy_class = 0
   if label_class_count != 0:
     probability_class = label_class_count/class_count
     entropy_class = - probability_class * np.log2(probability_class)
     entropy += entropy_class
 return entropy

In [ ]:
def calc_info_gain(feature_name, train_data, label, class_list):
 feature_value_list = train_data[feature_name].unique()
 total_row = train_data.shape[0]
 feature_info = 0.0

 for feature_value in feature_value_list:
   feature_value_data = train_data[train_data[feature_name] == feature_value]
   feature_value_count = feature_value_data.shape[0]
   feature_value_entropy = calc_entropy(feature_value_data,label, class_list)
   feature_value_probability = feature_value_count/total_row
   feature_info += feature_value_probability * feature_value_entropy

 return calc_total_entropy(train_data, label, class_list) - feature_info

In [ ]:
def find_most_informative_feature(train_data, label, class_list):
 feature_list = train_data.columns.drop(label)
 max_info_gain = -1
 max_info_feature = None

 for feature in feature_list:
   feature_info_gain = calc_info_gain(feature, train_data,label, class_list)
   if max_info_gain < feature_info_gain:
     max_info_gain = feature_info_gain
     max_info_feature = feature
 return max_info_feature

In [ ]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False)
    tree = {}

    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value]

        assigned_to_node = False
        for c in class_list:
            class_count = feature_value_data[feature_value_data[label] == c].shape[0]

            if class_count == count:
                tree[feature_value] = c
                train_data = train_data[train_data[feature_name] != feature_value]
                assigned_to_node = True
        if not assigned_to_node:
            tree[feature_value] = "?"

    return tree, train_data

In [ ]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0:
        max_info_feature = find_most_informative_feature(train_data, label, class_list)
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list)
        next_root = None

        if prev_feature_value != None:
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else:
            root[max_info_feature] = tree
            next_root = root[max_info_feature]

        for node, branch in list(next_root.items()):
            if branch == "?":
                feature_value_data = train_data[train_data[max_info_feature] == node]
                make_tree(next_root, node, feature_value_data, label, class_list)

In [ ]:
def id3(df2, label):
 train_data = df.copy()
 tree = {}
 class_list = train_data[label].unique()
 make_tree(tree, None, train_data, label, class_list)
 return tree

In [ ]:
def predict(tree, instance):
 if not isinstance(tree, dict):
   return tree
 else:
   root_node = next(iter(tree))
   feature_value = instance[root_node]
   if feature_value in tree[root_node]:
     return predict(tree[root_node][feature_value], instance)
   else:
     return None

In [ ]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows():
        result = predict(tree, test_data_m.iloc[index])
        if result == test_data_m[label].iloc[index]:
            correct_preditct += 1
        else:
            wrong_preditct += 1
    accuracy = correct_preditct / (correct_preditct + wrong_preditct)
    return accuracy

In [ ]:
tree1 = id3(df2, 'Terinfeksi')
tree1

KeyError: ignored

In [ ]:
evaluate(tree1, df2, 'Terinfeksi')